# ANNOTATED ENCODER-DECODER WITH ATTENTION - INFERENCING

Load the built model and translate German text to English


In [1]:
# mount gdrive
mount_drive = True
if mount_drive:
    from google.colab import drive
    drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir('/content/gdrive/My Drive/TSAI/EVA4_Phase2/session11-GRU-Attention-Transformers/notebooks/')
print(os.getcwd())

/content/gdrive/My Drive/TSAI/EVA4_Phase2/session11-GRU-Attention-Transformers/notebooks


In [3]:
# Import standard packages
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# To autoreload all te custom files when modified
import autoreload
%load_ext autoreload
%autoreload

In [5]:
from pathlib import Path

# important folders used in this application

SOLUTION_LOG_DIR = f"logs/logs_tmp"   
MODEL_BASEPATH = Path(f'./{SOLUTION_LOG_DIR}/saved_models') 

In [6]:
from models.attention_net import EncoderDecoder, Generator, Encoder, Decoder, BahdanauAttention

model_path = f'{MODEL_BASEPATH}/attention_cuda.pt'
model = torch.load(model_path, map_location=torch.device('cpu'))
model = model.to('cpu')
model.eval()
print('')

In [7]:
get_model_size = lambda filename: os.path.getsize(filename)/1e6

In [8]:
print(f'Pytorch model size (MB): {get_model_size(model_path):0.2f}')

Pytorch model size (MB): 51.45


In [9]:
#!pip install git+git://github.com/pytorch/text spacy 
#!python -m spacy download en
#!python -m spacy download de

In [10]:
# For data loading.
from torchtext import data, datasets

if True:
    import spacy
    spacy_de = spacy.load('de')
    spacy_en = spacy.load('en')

    def tokenize_de(text):
        return [tok.text for tok in spacy_de.tokenizer(text)]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]

    UNK_TOKEN = "<unk>"
    PAD_TOKEN = "<pad>"    
    SOS_TOKEN = "<s>"
    EOS_TOKEN = "</s>"
    LOWER = True
    
    # we include lengths to provide to the RNNs
    SRC = data.Field(tokenize=tokenize_de, 
                     batch_first=True, lower=LOWER, include_lengths=True,
                     unk_token=UNK_TOKEN, pad_token=PAD_TOKEN, init_token=None, eos_token=EOS_TOKEN)
    TRG = data.Field(tokenize=tokenize_en, 
                     batch_first=True, lower=LOWER, include_lengths=True,
                     unk_token=UNK_TOKEN, pad_token=PAD_TOKEN, init_token=SOS_TOKEN, eos_token=EOS_TOKEN)

    MAX_LEN = 25  # NOTE: we filter out a lot of sentences for speed
    train_data, valid_data, test_data = datasets.IWSLT.splits(
        exts=('.de', '.en'), fields=(SRC, TRG), 
        filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
            len(vars(x)['trg']) <= MAX_LEN)
    MIN_FREQ = 5  # NOTE: we limit the vocabulary to frequent words for speed
    SRC.build_vocab(train_data.src, min_freq=MIN_FREQ)
    TRG.build_vocab(train_data.trg, min_freq=MIN_FREQ)
    
    PAD_INDEX = TRG.vocab.stoi[PAD_TOKEN]


In [11]:
def greedy_decode(model, src, src_mask, src_lengths, max_len=100, sos_index=1, eos_index=None):
    """Greedily decode a sentence."""

    with torch.no_grad():
        encoder_hidden, encoder_final = model.encode(src, src_mask, src_lengths)
        prev_y = torch.ones(1, 1).fill_(sos_index).type_as(src)
        trg_mask = torch.ones_like(prev_y)

    output = []
    attention_scores = []
    hidden = None

    for i in range(max_len):
        with torch.no_grad():
            out, hidden, pre_output = model.decode(
              encoder_hidden, encoder_final, src_mask,
              prev_y, trg_mask, hidden)

            # we predict from the pre-output layer, which is
            # a combination of Decoder state, prev emb, and context
            prob = model.generator(pre_output[:, -1])

        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data.item()
        output.append(next_word)
        prev_y = torch.ones(1, 1).type_as(src).fill_(next_word)
        attention_scores.append(model.decoder.attention.alphas.cpu().numpy())
    
    output = np.array(output)
        
    # cut off everything starting from </s> 
    # (only when eos_index provided)
    if eos_index is not None:
        first_eos = np.where(output==eos_index)[0]
        if len(first_eos) > 0:
            output = output[:first_eos[0]]      
    
    return output, np.concatenate(attention_scores, axis=1)
  

def lookup_words(x, vocab=None):
    if vocab is not None:
        x = [vocab.itos[i] for i in x]

    return [str(t) for t in x]

In [12]:
class Batch:
    """Object for holding a batch of data with mask during training.
    Input is a batch from a torch text iterator.
    """
    def __init__(self, src, pad_index=0):
        
        src, src_lengths = src
        
        self.src = src
        self.src_lengths = src_lengths
        self.src_mask = (src != pad_index).unsqueeze(-2)
        self.nseqs = src.size(0)
        self.ntokens = None

In [13]:
def createBatch(inp_str, vocab, pad_idx):
    num_words=len(SRC.vocab)
    words = inp_str.split()
    wordIndexes = np.array([vocab.stoi[word] for word in words])
    src = torch.from_numpy(wordIndexes).unsqueeze(0)
    src_lengths = [len(wordIndexes)] * 1
    return Batch((src, src_lengths), pad_idx)

In [14]:
german_inp1 = "als ich 11 jahre alt war , wurde ich eines morgens von den <unk> heller freude geweckt"
german_inp2 = "mein vater hörte sich auf seinem kleinen , grauen radio die <unk> der bbc an ."

input = german_inp1
batch = createBatch(input, vocab = SRC.vocab, pad_idx=PAD_INDEX)

In [15]:
pred, attention = greedy_decode(
                          model, 
                          batch.src, batch.src_mask, batch.src_lengths, 
                          max_len=MAX_LEN,
                          sos_index=TRG.vocab.stoi[SOS_TOKEN],
                          eos_index=TRG.vocab.stoi[EOS_TOKEN])

In [16]:
output = " ".join(lookup_words(pred, vocab=TRG.vocab))

print("Input[German]  : ", input)
print("Output[English]: ", output)

Input[German]  :  als ich 11 jahre alt war , wurde ich eines morgens von den <unk> heller freude geweckt
Output[English]:  when i was 11 , i was able to get some of the morning <unk> myself in the morning of the morning .
